Методы отбора признаков
- RFE  
sklearn.feature_selection.RFE
- PCA  
- Tree-based  
sklearn.ensemble.ExtraTreesClassifier
- SVM + SelectFromModel  
sklearn.svm.SVC
sklearn.feature_selection.SelectFromModel


roc auc score надо посчитать

https://www.kaggle.com/competitions/playground-series-s3e22/data?select=train.csv

In [41]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)

In [42]:
path = 'data/train.csv'
hourse_data = pd.read_csv(path, index_col=0)

In [43]:
hourse_data.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,NaN,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived


- surgery
  - Yes, it had surgery
  - It was treated without surgery
- Age
  - Adult horse
  - Young (< 6 months)
- Hospital Number
  - numeric id
  - the case number assigned to the horse (may not be unique if the horse is treated > 1 time)
- rectal temperature
  - linear
  - in degrees celsius
  - An elevated temp may occur due to infection
  - temperature may be reduced when the animal is in late shock
  - normal temp is 37.8
  - this parameter will usually change as the problem progresses, eg. may start out normal, then become elevated because of the lesion, passing back through the normal range as the horse goes into shock 
- pulse
  - linear
  - the heart rate in beats per minute
  - is a reflection of the heart condition: 30 -40 is normal for adults
  - rare to have a lower than normal rate although athletic horses may have a rate of 20-25 
  - animals with painful lesions or suffering from circulatory shock may have an elevated heart rate 
- respiratory rate 
  - linear
  - normal rate is 8 to 10
  - usefulness is doubtful due to the great fluctuations
- temperature of extremities (cool to cold extremities indicate possible shock)
  - Normal 
  - Warm 
  - Cool 
  - Cold
- peripheral pulse
  - normal 
  - increased 
  - reduced 
  - absent
- mucous membranes
  - possible values:
    - normal pink 
    - bright pink 
    - pale pink 
    - pale cyanotic 
    - bright red / injected 
    - dark cyanotic
  - 1 and 2 probably indicate a normal or slightly increased circulation 
  - 3 may occur in early shock 
  - 4 and 6 are indicative of serious circulatory compromise 
  - 5 is more indicative of a septicemia
- capillary refill time 
  - a clinical judgement. The longer the refill, the poorer the circulation
  - possible values: 
    - more than 3 sec
    - 3 sec exactly
    - less than 3 sec
- pain - a subjective judgement of the horse's pain level
  - possible values: 
    - alert, no pain 
    - depressed 
    - intermittent mild pain 
    - intermittent severe pain 
    - continuous severe pain
  - should NOT be treated as a ordered or discrete variable
  - In general, the more painful, the more likely it is to require surgery
  - prior treatment of pain may mask the pain level to some extent
- peristalsis
  - an indication of the activity in the horse's gut. As the gut becomes more distended or the horse becomes more toxic, the activity decreases 
  - possible values:
    - hypermotile 
    - normal 
    - hypomotile 
    - absent
    - distend_small
- abdominal distension
  - An IMPORTANT parameter
  - an animal with abdominal distension is likely to be painful and have reduced gut motility
  - a horse with severe abdominal distension is likely to require surgery just tio relieve the pressure
  - possible values:
    - none 
    - slight 
    - moderate 
    - severe
- nasogastric tube
  - this refers to any gas coming out of the tube
  - a large gas cap in the stomach is likely to give the horse discomfort
    - none 
    - slight 
    - significant
- nasogastric reflux
  - the greater amount of reflux, the more likelihood that there is some serious obstruction to the fluid passage from the rest of the intestine 
  - possible values:
    - none
    - more than 1 liter
    - less than 1 liter
- nasogastric reflux PH
  - linear
  - scale is from 0 to 14 with 7 being neutral
  - normal values are in the 3 to 4 range
- rectal examination - feces
  - absent feces probably indicates an obstruction
  - possible values:
    - normal 
    - increased 
    - decreased 
    - absent
- abdomen
  - possible values:
    - normal 
    - other 
    - firm feces in the large intestine 
    - distended small intestine 
    - distended large intestine
  - 3 is probably an obstruction caused by a mechanical impaction and is normally treated medically 
  - 4 and 5 indicate a surgical lesion 
- packed cell volume
  - linear
  - the # of red cells by volume in the blood
  - normal range is 30 to 50. The level rises as the circulation becomes compromised or as the animal becomes dehydrated. 
- total protein
  - linear
  - normal values lie in the 6-7.5 (gms/dL) range 
  - the higher the value the greater the dehydration 
- abdominocentesis appearance (abdomo_appearance)
  - a needle is put in the horse's abdomen and fluid is obtained from the abdominal cavity
  - normal fluid is clear while cloudy or serosanguinous indicates a compromised gut 
  - possible values: 
    - clear 
    - cloudy 
    - serosanguinous 
- abdomcentesis total protein
  - linear
  - the higher the level of protein the more likely it is to have a compromised gut. Values are in gms/dL 
- surgical lesion
  - possible values: 
    - Yes 
    - No
- leison types:
  - first number is site of lesion 
    -  1 = gastric 
    -  2 = sm intestine 
    -  3 = lg colon 
    -  4 = lg colon and cecum 
    -  5 = cecum 
    -  6 = transverse colon 
    -  7 = retum/descending colon 
    -  8 = uterus
    -  9 = bladder
    -  11 = all intestinal sites 
    -  00 = none
  - second number is type 
    -  1 = simple 
    -  2 = strangulation 
    -  3 = inflammation 
    -  4 = other
  - third number is subtype 
    - 1 = mechanical 
    - 2 = paralytic 
    - 0 = n/a 
  - fourth number is specific code 
    - 1 = obturation 
    - 2 = intrinsic
    - 3 = extrinsic
    - 4 = adynamic
    - 5 = volvulus/torsion
    - 6 = intussuption 
    - 7 = thromboembolic
    - 8 = hernia
    - 9 = lipoma/slenic incarceration 
    - 10 = displacement 
    - 0 = n/a
    - 28: cp_data
- is pathology data present for this case? (cp_data) 
  - Yes 
  - No 
- outcome
  - what eventually happened to the horse? 
  - possible values:
    - lived 
    - died 
    - was euthanized

In [44]:
categories = {
    'surgery': ['yes', 'no'],
    'age': ['adult', 'young'],
    'temp_of_extremities': ['Normal', 'Warm', 'Cool', 'Cold'],
    'peripheral_pulse': ['normal', 'increased', 'reduced', 'absent'],
    'mucous_membrane': ['normal_pink', 'bright_pink', 'pale_pink', 'pale_cyanotic', 'bright_red', 'dark_cyanotic'],
    'capillary_refill_time': ['3', 'less_3_sec', 'more_3_sec'],
    'pain': ['depressed', 'mild_pain', 'alert', 'severe_pain', 'extreme_pain'],     # slight - 1 значение
    'peristalsis': ['hypermotile', 'normal', 'hypomotile', 'absent'],               # distend_small - 1 значение
    'abdominal_distention': ['none', 'slight', 'moderate', 'severe'],
    'nasogastric_tube': ['none', 'slight', 'significant'],
    'nasogastric_reflux': ['more_1_liter', 'none', 'less_1_liter'],                 # slight - 1 значение
    'rectal_exam_feces': ['normal', 'increased', 'decreased', 'absent'],            # serosanguious - 1 значение
    'abdomen': ['normal', 'other', 'firm', 'distend_small', 'distend_large'],
    'abdomo_appearance': ['clear', 'cloudy', 'serosanguious'],
    'surgical_lesion': ['yes', 'no'],
    'cp_data': ['yes', 'no'],
    'outcome': ['lived', 'died', 'was euthanized']
}
categorical_columns = ['surgery', 'age', 'temp_of_extremities', 'peripheral_pulse', 'mucous_membrane', 
                       'capillary_refill_time', 'pain', 'peristalsis', 'abdominal_distention', 'nasogastric_tube',
                       'nasogastric_reflux', 'rectal_exam_feces', 'abdomen', 'abdomo_appearance', 'surgical_lesion',
                       'cp_data']

In [45]:
def encode_data(df, encode_features):
    new_df = pd.get_dummies(df, columns=encode_features, dtype='int64')
    return new_df

mask = (hourse_data['pain'] == 'slight') |\
       (hourse_data['peristalsis'] == 'distend_small') |\
       (hourse_data['nasogastric_reflux'] == 'slight') |\
       (hourse_data['rectal_exam_feces'] == 'serosanguious')

print(hourse_data.shape)
row_df = hourse_data.drop(index=hourse_data[mask].index)
#row_df = row_df.drop(columns=['lesion_1', 'lesion_2', 'lesion_3'])
print(row_df.shape)


print(row_df.shape)
row_df = pd.get_dummies(row_df, columns=categorical_columns, dtype='int64')
print(row_df.shape)

(1235, 28)
(1231, 28)
(1231, 28)
(1231, 68)


In [46]:
row_df

,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3,outcome,surgery_no,surgery_yes,age_adult,age_young,temp_of_extremities_cold,temp_of_extremities_cool,temp_of_extremities_normal,temp_of_extremities_warm,peripheral_pulse_absent,peripheral_pulse_increased,peripheral_pulse_normal,peripheral_pulse_reduced,mucous_membrane_bright_pink,mucous_membrane_bright_red,mucous_membrane_dark_cyanotic,mucous_membrane_normal_pink,mucous_membrane_pale_cyanotic,mucous_membrane_pale_pink,capillary_refill_time_3,capillary_refill_time_less_3_sec,capillary_refill_time_more_3_sec,pain_alert,pain_depressed,pain_extreme_pain,pain_mild_pain,pain_severe_pain,peristalsis_absent,peristalsis_hypermotile,peristalsis_hypomotile,peristalsis_normal,abdominal_distention_moderate,abdominal_distention_none,abdominal_distention_severe,abdominal_distention_slight,nasogastric_tube_none,nasogastric_tube_significant,nasogastric_tube_slight,nasogastric_reflux_less_1_liter,nasogastric_reflux_more_1_liter,nasogastric_reflux_none,rectal_exam_feces_absent,rectal_exam_feces_decreased,rectal_exam_feces_increased,rectal_exam_feces_normal,abdomen_distend_large,abdomen_distend_small,abdomen_firm,abdomen_normal,abdomen_other,abdomo_appearance_clear,abdomo_appearance_cloudy,abdomo_appearance_serosanguious,surgical_lesion_no,surgical_lesion_yes,cp_data_no,cp_data_yes
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,530001,38.1,132.0,24.0,6.5,57.0,8.5,3.4,2209,0,0,died,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,0
1,533836,37.5,88.0,12.0,2.0,33.0,64.0,2.0,2208,0,0,euthanized,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0
2,529812,38.3,120.0,28.0,3.5,37.0,6.4,3.4,5124,0,0,lived,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0
3,5262541,37.1,72.0,30.0,2.0,53.0,7.0,3.9,2208,0,0,lived,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1
4,5299629,38.0,52.0,48.0,7.0,47.0,7.3,2.6,0,0,0,lived,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,535246,38.5,129.0,48.0,2.0,57.0,66.0,2.0,2206,0,0,lived,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0
1231,528570,37.5,60.0,50.0,3.0,35.0,6.4,3.6,2209,0,0,died,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1
1232,529685,37.5,84.0,40.0,3.0,40.0,5.9,7.0,400,0,0,lived,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1


In [47]:
# В признаках lesion_2 и lesion_3 практически все показатели равны нулю, поэтому они не информативны и подлежат удалению
print(hourse_data['lesion_2'].value_counts())
print(hourse_data['lesion_3'].value_counts())
row_df = row_df.drop(columns=['lesion_2', 'lesion_3'])

lesion_2
0       1227
1400       4
3111       2
3112       2
Name: count, dtype: int64
lesion_3
0       1233
2209       2
Name: count, dtype: int64


In [57]:
# Запишем цифровые метки и их значения в виде словаря для признака leison
lesion_first_number = {
    "1": "gastric",  "2": "sm_intestine", "3": "lg_colon",
    "4": "lg_colon_and_cecum", "5": "cecum", "6": "transverse_colon",
    "7": "retum/descending_colon", "8": "uterus", "9": "bladder",
    "11": "all_intestinal_sites", "00": "none",
}
lesion_second_number = {
    "1": "simple", "2": "strangulation",
    "3": "inflammation", "4": "other",
    "0": "none",
}
lesion_third_number = {
    "1": "mechanical", "2": "paralytic", "0": "none",
}
lesion_fourth_number = {
    "1": "obturation", "2": "intrinsic", "3": "extrinsic",
    "4": "adynamic", "5": "volvulus/torsion", "6": "intussuption",
    "7": "thromboembolic", "8": "hernia", "9": "lipoma/slenic_incarceration",
    "10": "displacement", "0": "none",
}

# Пишем функцию, разделяющую цифровой набор симптомов на составляющие части
def lesion_splited(key):
    if key[:2] in ['00', '11']:
        first, second, third, fourth = key[:2], key[2], key[3], key[4:]
    elif key[0] == '0':
        first, second, third, fourth = 'none', 'none', 'none', 'none'
    else:
        first, second, third, fourth = key[0], key[1], key[2], key[3:]

    return [ lesion_first_number.get(first, "none"), lesion_second_number.get(second, "none"), 
            lesion_third_number.get(third, "none"), lesion_fourth_number.get(fourth, "none")]
        
# Заменяем цифровой набор симптомов на словесный список
row_df['lesion_1'] = row_df['lesion_1'].astype('string').apply(lesion_splited)

In [65]:
# Ключевые слова для преобразования для one hot encoding
lesion_keys = [
    "gastric",  "sm_intestine", "lg_colon",
    "lg_colon_and_cecum", "cecum", "transverse_colon",
    "retum/descending_colon", "uterus", "bladder",
    "all_intestinal_sites", 
    "simple", "strangulation",
    "inflammation", "other", "mechanical", "paralytic",
    "obturation", "intrinsic", "extrinsic",
    "adynamic", "volvulus/torsion", "intussuption",
    "thromboembolic", "hernia", "lipoma/slenic_incarceration",
    "displacement"]

# Преобразуем признак lesion_1 по ключам по циклу и удаляем исходный столбец
for key in lesion_keys:
    row_df[key] = row_df['lesion_1'].apply(lambda x: 1 if key in x else 0)
    
row_df.drop(columns=['lesion_1'], axis=1, inplace=True)

display(row_df.head())
print(row_df.shape)

,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,outcome,surgery_no,surgery_yes,age_adult,age_young,temp_of_extremities_cold,temp_of_extremities_cool,temp_of_extremities_normal,temp_of_extremities_warm,peripheral_pulse_absent,peripheral_pulse_increased,peripheral_pulse_normal,peripheral_pulse_reduced,mucous_membrane_bright_pink,mucous_membrane_bright_red,mucous_membrane_dark_cyanotic,mucous_membrane_normal_pink,mucous_membrane_pale_cyanotic,mucous_membrane_pale_pink,capillary_refill_time_3,capillary_refill_time_less_3_sec,capillary_refill_time_more_3_sec,pain_alert,pain_depressed,pain_extreme_pain,pain_mild_pain,pain_severe_pain,peristalsis_absent,peristalsis_hypermotile,peristalsis_hypomotile,peristalsis_normal,abdominal_distention_moderate,abdominal_distention_none,abdominal_distention_severe,abdominal_distention_slight,nasogastric_tube_none,nasogastric_tube_significant,nasogastric_tube_slight,nasogastric_reflux_less_1_liter,nasogastric_reflux_more_1_liter,nasogastric_reflux_none,rectal_exam_feces_absent,rectal_exam_feces_decreased,rectal_exam_feces_increased,rectal_exam_feces_normal,abdomen_distend_large,abdomen_distend_small,abdomen_firm,abdomen_normal,abdomen_other,abdomo_appearance_clear,abdomo_appearance_cloudy,abdomo_appearance_serosanguious,surgical_lesion_no,surgical_lesion_yes,cp_data_no,cp_data_yes,gastric,sm_intestine,lg_colon,lg_colon_and_cecum,cecum,transverse_colon,retum/descending_colon,uterus,bladder,all_intestinal_sites,simple,strangulation,inflammation,other,mechanical,paralytic,obturation,intrinsic,extrinsic,adynamic,volvulus/torsion,intussuption,thromboembolic,hernia,lipoma/slenic_incarceration,displacement
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,530001,38.1,132.0,24.0,6.5,57.0,8.5,3.4,died,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,533836,37.5,88.0,12.0,2.0,33.0,64.0,2.0,euthanized,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,529812,38.3,120.0,28.0,3.5,37.0,6.4,3.4,lived,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,5262541,37.1,72.0,30.0,2.0,53.0,7.0,3.9,lived,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,5299629,38.0,52.0,48.0,7.0,47.0,7.3,2.6,lived,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


(1231, 91)


In [85]:
grouped = row_df.groupby(by=['hospital_number'])['outcome'].sum()
display(grouped)
print(row_df['hospital_number'].nunique())
# Всего случаев лечения 255 из 1231, лошади лечились несколько раз

hospital_number
521399                dieddiedeuthanized
522979                              died
523190                   livedeuthanized
526639                         diedlived
526802                    liveddiedlived
                       ...              
5299253                        liveddied
5299603                  livedlivedlived
5299629    liveddiedlivedlivedlivedlived
5301219             dieddieddieddieddied
5305129                            lived
Name: outcome, Length: 255, dtype: object

255


In [86]:
row_df.duplicated().sum()       # дупликаты отсутствуют

0

In [100]:
import matplotlib.pyplot as plt
import plotly.express as px
# Графики признаков с выбросами
fig = px.box(
    data_frame=row_df,
    x='total_protein',
    y='outcome',
    height=400,
    width=600,
    title='total_protein boxplot'
);
fig.show()

In [110]:
fig = px.box(
    data_frame=row_df,
    x='respiratory_rate',
    height=400,
    width=600,
    title='respiratory_rate boxplot'
);
fig.show()

In [113]:
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [126]:
y = row_df['outcome']
X = row_df.drop(columns=['outcome', 'hospital_number'])

In [127]:
# Корреляционная матрица признаков
corr = X.corr()
display(corr)

,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,surgery_no,surgery_yes,age_adult,age_young,temp_of_extremities_cold,temp_of_extremities_cool,temp_of_extremities_normal,temp_of_extremities_warm,peripheral_pulse_absent,peripheral_pulse_increased,peripheral_pulse_normal,peripheral_pulse_reduced,mucous_membrane_bright_pink,mucous_membrane_bright_red,mucous_membrane_dark_cyanotic,mucous_membrane_normal_pink,mucous_membrane_pale_cyanotic,mucous_membrane_pale_pink,capillary_refill_time_3,capillary_refill_time_less_3_sec,capillary_refill_time_more_3_sec,pain_alert,pain_depressed,pain_extreme_pain,pain_mild_pain,pain_severe_pain,peristalsis_absent,peristalsis_hypermotile,peristalsis_hypomotile,peristalsis_normal,abdominal_distention_moderate,abdominal_distention_none,abdominal_distention_severe,abdominal_distention_slight,nasogastric_tube_none,nasogastric_tube_significant,nasogastric_tube_slight,nasogastric_reflux_less_1_liter,nasogastric_reflux_more_1_liter,nasogastric_reflux_none,rectal_exam_feces_absent,rectal_exam_feces_decreased,rectal_exam_feces_increased,rectal_exam_feces_normal,abdomen_distend_large,abdomen_distend_small,abdomen_firm,abdomen_normal,abdomen_other,abdomo_appearance_clear,abdomo_appearance_cloudy,abdomo_appearance_serosanguious,surgical_lesion_no,surgical_lesion_yes,cp_data_no,cp_data_yes,gastric,sm_intestine,lg_colon,lg_colon_and_cecum,cecum,transverse_colon,retum/descending_colon,uterus,bladder,all_intestinal_sites,simple,strangulation,inflammation,other,mechanical,paralytic,obturation,intrinsic,extrinsic,adynamic,volvulus/torsion,intussuption,thromboembolic,hernia,lipoma/slenic_incarceration,displacement
rectal_temp,1.000000,0.174349,0.125991,0.122098,0.090972,-0.058987,0.041916,0.057496,-0.057496,-0.062729,0.062729,0.045230,0.018937,-0.052733,0.013468,0.089916,0.017893,-0.027259,0.013384,0.001778,0.066174,0.073740,0.017899,-0.081571,-0.054187,-0.041331,-0.101687,0.101636,-0.010352,0.162596,-0.074638,-0.064995,-0.054404,-0.057131,-0.031186,0.074557,-0.012827,0.044428,-0.005374,0.021861,-0.060804,0.038726,-0.072195,0.005166,0.001921,-0.121029,0.132171,-0.013944,-0.048129,-0.007407,0.048234,-0.012886,-0.013048,-0.033932,0.014585,-0.024264,-0.023774,-0.002631,0.037997,0.020527,-0.020527,0.010822,-0.010822,-0.077490,-0.020526,0.032156,-0.021000,-0.045837,-0.046795,0.130379,0.042682,0.009434,-0.026863,0.046908,-0.051554,-0.005761,0.020054,0.031923,0.006090,-0.005808,-0.060319,0.109389,0.036635,0.023210,0.025767,-0.034673,-0.024282,-0.058396,0.017499
pulse,0.174349,1.000000,0.396489,-0.006888,0.435839,0.003332,0.055422,-0.133241,0.133241,-0.279980,0.279980,0.220744,0.187821,-0.322043,-0.109217,0.068148,-0.047284,-0.438024,0.443744,-0.001003,0.044733,0.287208,-0.335837,0.171373,-0.051995,0.000547,-0.447238,0.450643,-0.263529,0.370604,0.134868,-0.308707,-0.013410,0.323564,-0.151529,-0.159349,-0.207627,0.222706,-0.217175,0.169816,-0.165481,0.107713,-0.036942,-0.114520,-0.050166,0.010164,0.051178,0.131970,-0.046405,-0.054626,-0.194285,0.036334,0.064404,-0.166925,-0.057862,-0.119331,-0.279223,-0.177887,0.370037,-0.233344,0.233344,0.107359,-0.107359,0.100071,0.099766,-0.063820,0.035889,-0.044718,0.010822,0.066072,0.023502,0.060543,0.045631,-0.155943,0.219466,0.024454,0.124313,-0.154633,-0.021000,-0.165778,-0.037361,0.022476,-0.024001,0.180993,0.012683,0.080090,0.057857,0.008461,-0.059386
respiratory_rate,0.125991,0.396489,1.000000,0.003568,0.189694,0.005981,0.004622,-0.108607,0.108607,-0.190145,0.190145,0.133709,0.072274,-0.139924,-0.017128,0.103790,0.005063,-0.182902,0.183038,0.020580,0.096938,0.216158,-0.188753,0.009448,-0.068671,-0.012365,-0.188826,0.198253,-0.139994,0.208181,0.040300,-0.170589,0.022295,0.130271,-0.090319,-0.023909,-0.147255,0.093696,-0.088982,0.142842,-0.115290,0.085683,0.010661,-0.066892,0.118922,-0.065995,-0.025385,0.065701,-0.019041,-0.053564,-0.077117,-0.006615,0.033749,-0.133430,-0.000851,-0.069007,-0.151885,-0.055880,0.168198,-0.118821,0.1

In [132]:
fig = px.imshow(corr,
          title='Heatmap of features correlation',
          height=2000,
          width=2000)
fig.show()